# Feature selection with impurity and permutation importance on the Ames Housing dataset.
In this notebook, we will compare UFI, MDI and Permutation importance on their ability to perform feature selection on the high-dimensional Ames Housing dataset, and their associated computational cost.

# Import Libraries

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
import warnings

from sklearn.compose import make_column_transformer
from sklearn.datasets import fetch_openml
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.inspection import permutation_importance
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder

# Data fetching

In [2]:
ames_housing = fetch_openml(data_id=42165, as_frame=True, return_X_y=False)
y = ames_housing["target"].astype(float)
ames_housing = ames_housing["data"].drop(columns="Id")
ames_housing.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal


# Data preprocessing

In [3]:
numerical_features = ames_housing.select_dtypes("number").columns
categorical_features = ames_housing.columns.difference(numerical_features)

categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="most_frequent")),
    ('encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
])

preprocessor = make_column_transformer(
    (categorical_pipeline, categorical_features),
    (SimpleImputer(strategy="mean"), numerical_features),
    remainder='passthrough'
)

ames_housing_transformed = preprocessor.fit_transform(ames_housing)

feature_names = (
    categorical_features.tolist() + 
    numerical_features.tolist()
)

ames_housing_preprocessed = pd.DataFrame(
    ames_housing_transformed,
    columns=feature_names,
    index=ames_housing.index
)

# Add random features of varying cardinality

In [4]:
random_cat_sizes = [2, 5, 10, 20, 50, 100]
random_features = dict()
n_sample = len(y)
rng = np.random.RandomState(42)

X = ames_housing_preprocessed.copy()
for cat_size in random_cat_sizes:
    X[f"random_cat_{cat_size}"] = rng.randint(0, cat_size, size=n_sample)
X["random_num"] = rng.normal(size=n_sample)

X.head()

,Alley,BldgType,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,BsmtQual,CentralAir,Condition1,Condition2,...,MiscVal,MoSold,YrSold,random_cat_2,random_cat_5,random_cat_10,random_cat_20,random_cat_50,random_cat_100,random_num
0,0.0,0.0,3.0,3.0,2.0,5.0,2.0,1.0,2.0,2.0,...,0.0,2.0,2008.0,0,4,7,19,36,68,-1.583187
1,0.0,0.0,3.0,1.0,0.0,5.0,2.0,1.0,1.0,2.0,...,0.0,5.0,2007.0,1,4,0,17,45,53,0.114476
2,0.0,0.0,3.0,2.0,2.0,5.0,2.0,1.0,2.0,2.0,...,0.0,9.0,2008.0,0,4,4,9,23,53,1.573412
3,0.0,0.0,1.0,3.0,0.0,5.0,3.0,1.0,2.0,2.0,...,0.0,2.0,2006.0,0,2,2,8,15,27,0.558137
4,0.0,0.0,3.0,0.0,2.0,5.0,2.0,1.0,2.0,2.0,...,0.0,12.0,2008.0,0,2,0,7,12,55,-0.105790


# Train and optimize a `RandomForestRegressor`.

In [5]:
# Define parameter grid for min_samples_leaf
param_grid = {"min_samples_leaf": [1, 5, 20, 100], "max_features":["log2", "sqrt", 1.0], "n_estimators": [10, 50, 100, 200, 400]}

# Initialize Random Forest
rf = RandomForestRegressor(random_state=rng, n_jobs=-1)
rf.fit(X, y)

# Set up GridSearchCV with cross-validation
grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    cv=5,
    scoring="neg_mean_squared_error",
    return_train_score=True,
    n_jobs=-1,
    verbose=1,
)

# Fit the grid search
grid_search.fit(X, y)

# Get the best parameter and estimator
best_params = grid_search.best_params_
best_score_gs = grid_search.best_score_

print(f"Best params: {best_params}")
print(f"Best neg mse score: {best_score_gs:.3e}")

common_params = best_params
common_params["random_state"] = rng
common_params["n_jobs"] = -1



Fitting 5 folds for each of 60 candidates, totalling 300 fits
Best params: {'max_features': 'sqrt', 'min_samples_leaf': 1, 'n_estimators': 400}
Best neg mse score: -9.035e+08


# Feature selection procedure

In [6]:
def feature_selection(method, common_params, X, y, verbose=False):
    start_time = time.time()
    warnings.filterwarnings(
        "ignore", message=r"The number of unique classes is greater than 50% .*"
    )

    if method == "permutation":
        # Test data is required for reliable permutation importance
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=int(0.2 * X.shape[0]), random_state=rng
        )
        common_params["oob_score"] = False

    elif method == "UFI":
        X_train, y_train = X, y
        # With oob_score=True, rf will compute UFI during fit
        common_params["oob_score"] = True

    # Define a rf regressor
    rf = RandomForestRegressor(**common_params)

    # Compute neg mse score for the model trained on the dataset with all the features
    cv_score = cross_val_score(
        rf, X_train, y_train, scoring="neg_mean_squared_error", cv=5, n_jobs=-1
    )
    new_score = cv_score.mean()
    prev_score = new_score
    best_score = new_score
    tol = cv_score.std()
    init_score = new_score

    # We keep removing features until the best cross validated score is degraded by at least one std
    while new_score > best_score - tol:
        prev_score = new_score

        # Store the best score and associated features
        if new_score > best_score:
            best_score = new_score
            best_columns = X_train.columns.to_list()
            tol = cv_score.std()
            
        # Re-fit the model
        rf.fit(X_train, y_train)

        # Compute feature importance
        if method == "permutation":
            importance = permutation_importance(
                rf,
                X_test,
                y_test,
                scoring="neg_mean_squared_error",
                n_repeats=5,
                n_jobs=-1,
                random_state=rng,
            ).importances_mean
        elif method == "UFI":
            importance = rf.unbiased_feature_importances_

        # Find the feature with the smallest importance
        smallest_column = X_train.columns.to_list()[np.argmin(importance)]

        # Remove the feature from the dataset
        X_train = X_train.drop(columns=smallest_column)
        if method == "permutation":
            X_test = X_test.drop(columns=smallest_column)

        # Compute a 5-fold cross-validated score on the reduced dataset
        cv_score = cross_val_score(
            rf, X_train, y_train, scoring="neg_mean_squared_error", cv=5, n_jobs=-1
        )
        new_score = cv_score.mean()
        if verbose:
            print(
                f"Dropped column: {smallest_column:15} Previous score: {prev_score:.2e}, New score: {new_score:.2e}"
            )
    run_time = time.time() - start_time
    return dict(
        run_time=run_time,
        init_score=init_score,
        best_score=best_score,
        best_columns=best_columns,
        final_score=prev_score,
        final_columns=X_train.columns.to_list()+[smallest_column],
    )


# UFI for feature selection
We will use the importance given by UFI under `feature_importances_` to perform feature selection.

In [7]:
res_UFI = feature_selection("UFI", common_params, X, y, verbose=True)
print(
    f"UFI results:\n"
    f"Removed {len(X.columns) - len(res_UFI["final_columns"])} features in {res_UFI["run_time"]:6.1f} seconds. \n"
    f"Initial score: {res_UFI["init_score"]:.2e}, best score: {res_UFI["best_score"]:.2e}, final score: {res_UFI["final_score"]:.2e} \n"
    f"Remaining columns: {res_UFI["final_columns"]}"
)

Dropped column: random_cat_20   Previous score: -9.03e+08, New score: -9.13e+08
Dropped column: random_cat_50   Previous score: -9.13e+08, New score: -8.81e+08
Dropped column: random_cat_2    Previous score: -8.81e+08, New score: -8.97e+08
Dropped column: random_num      Previous score: -8.97e+08, New score: -8.81e+08
Dropped column: random_cat_100  Previous score: -8.81e+08, New score: -8.79e+08
Dropped column: PoolArea        Previous score: -8.79e+08, New score: -8.86e+08
Dropped column: random_cat_10   Previous score: -8.86e+08, New score: -8.86e+08
Dropped column: LotConfig       Previous score: -8.86e+08, New score: -8.61e+08
Dropped column: YrSold          Previous score: -8.61e+08, New score: -8.71e+08
Dropped column: MoSold          Previous score: -8.71e+08, New score: -8.61e+08
Dropped column: Fence           Previous score: -8.61e+08, New score: -8.63e+08
Dropped column: PoolQC          Previous score: -8.63e+08, New score: -8.55e+08
Dropped column: GarageQual      Previous

In [8]:
res_permutation = feature_selection("permutation", common_params, X, y, verbose=True)
print(
    f"Permutation importance results:\n"
    f"Removed {len(X.columns) - len(res_permutation["final_columns"])} features in {res_permutation["run_time"]:6.1f} seconds. \n"
    f"Initial score: {res_permutation["init_score"]:.2e}, best score: {res_permutation["best_score"]:.2e}, final score: {res_permutation["final_score"]:.2e} \n"
    f"Remaining columns: {res_permutation["final_columns"]}"
)

Dropped column: MoSold          Previous score: -9.38e+08, New score: -9.36e+08
Dropped column: BsmtUnfSF       Previous score: -9.36e+08, New score: -9.32e+08
Dropped column: random_cat_100  Previous score: -9.32e+08, New score: -9.20e+08
Dropped column: random_cat_50   Previous score: -9.20e+08, New score: -9.32e+08
Dropped column: ExterCond       Previous score: -9.32e+08, New score: -9.09e+08
Dropped column: random_num      Previous score: -9.09e+08, New score: -9.25e+08
Dropped column: Exterior1st     Previous score: -9.25e+08, New score: -9.12e+08
Dropped column: LandSlope       Previous score: -9.12e+08, New score: -9.30e+08
Dropped column: FireplaceQu     Previous score: -9.30e+08, New score: -9.18e+08
Dropped column: BsmtHalfBath    Previous score: -9.18e+08, New score: -9.08e+08
Dropped column: YrSold          Previous score: -9.08e+08, New score: -9.13e+08
Dropped column: BsmtFinType2    Previous score: -9.13e+08, New score: -9.18e+08
Dropped column: random_cat_5    Previous